# AUTOGRAD: 자동 미분

pytorch의 모든 신경망 중심에는 `AUTOGRAD` 패키지가 있습니다. 'AUTOGRAD' 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다.
이는 실행-기반 정의(`define-by-run`) 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라집니다.

# Tensor

패키지의 중심에는 `torch.Tensor` 클래스가 있습니다. 만약 `.requires_grad` 속성을 `True`로 설정하면, 해당 Tensor에서 이뤄진 모든 연산들을 추적(Track)하기 시작합니다.
계산이 완료된 후, `.backward()`를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있습니다. 해당 Tensor의 변화도는 `.grad` 속성에 누적됩니다.

Tensor가 기록 추적하는 것을 중단하게 하려면, `.detach()` 를 호출하여 연산 기록으로부터 분리(detach)합니다.

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을 `with torch.no_grad():` 로 감쌀 수 있습니다.
이는 특히 변화도(gradient)는 필요없지만, requires_grad=True 가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(Evaluate)할 때 유용합니다.

Autogard 구현에서 매우 중요한 클래스가 하나 더 있는데, `Function` 클래스입니다.

`Tensor`와 `Function`은 서로 연결되어 있으며, 모든 연산 과정을 부호화(Encode)하여 순환하지 않은 그래프(acyclic graph)를 생성합니다.
각 tensor는 `.grad_fn` 속성을 갖고 있는데, 이는 `Tensor`를 생성한 `Function`을 참조하고 있습니다. (단, 사용자가 만든 Tensor는 예외로, `grad_fn=None`)

도함수를 계산하기 위해서는 `Tensor`의 `.backward()`를 호출하면 됩니다. 만약 `Tensor`가 스칼라(scalar)인 경우(ex. 하나의 요소 값만 갖는 경우)에는 `backward`에 인자를 정해줄 필요가 없습니다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 `gradient`의 인자로 지정할 필요가 있습니다.

In [16]:
import torch

tensor를 생성하고 `requires_grad=True`를 설정하여 연산을 기록합니다.

In [17]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Tensor에 연산을 수행합니다.

In [18]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`는 연산의 결과로 생성된 것이므로 `grad_fn`을 갖습니다.

In [19]:
print(y.grad_fn)

`y`에 다른 연산을 수행합니다.

In [20]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)` 는 기존 Tensor의 `requires_grad` 값을 바꿔치기(in-place)하여 변경합니다. 입력값이 지정되지 않으면 기본값은 `False` 입니다.

In [21]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

False


In [22]:
a.requires_grad_(True)

tensor([[65.0804, -2.2994],
        [-1.1172,  0.1594]], requires_grad=True)

In [23]:
b = (a * a).sum()
print(b)

tensor(4242.0127, grad_fn=<SumBackward0>)


# 변화도(Gradient)

역전파를 통해 변화도를 전파합니다. `out`은 하나의 스칼라 값만 갖고 있기 떄문에, `out_.backward()`는 `out.backward(torch.tensor(1.))` 과 동일합니다.

In [24]:
out.backward()

In [25]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



일반적으로 `torch.autograd`는 벡터-아코비안 곱을 계산하는 엔진입니다.

즉, 어떤 벡터 𝑣=(𝑣1𝑣2⋯𝑣𝑚)𝑇 에 대해 𝑣𝑇⋅𝐽 을 연산합니다.

만약 𝑣 가 스칼라 함수  𝑙=𝑔(𝑦⃗ ) 의 기울기인 경우, 𝑣=(∂𝑙∂𝑦1⋯∂𝑙∂𝑦𝑚)𝑇 이며, 연쇄법칙(chain rule)에 다라 벡터-아코비안 곱은  𝑥⃗ 에 대한 𝑙 의 기울기가 됩니다.

(여기서 𝑣𝑇⋅𝐽 은 𝐽𝑇⋅𝑣 를 취했을 때의 열 벡터로 취급할 수 있는 행 벡터를 갖습니다.)

벡터-야코비안 곱의 이러한 특성은 스칼라가 아닌 출력을 갖는 모델에 외부 변화도를 제공(feed)하는 것을 매우 편리하게 해줍니다.

In [26]:
x = torch.randn(3, requires_grad=True)

y = x * 2

while y.data.norm() < 1000:
    y *= 2

print(y)

tensor([ -442.3408, -1297.2008,  -764.4599], grad_fn=<MulBackward0>)


In [30]:
v = torch.tensor([.1, 1., 1e-4], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


또한 `with torch.no_grad():`로 코드 블록을 감싸서 autograd가 `.requires_grad=True`인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있습니다.

In [31]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


또는 `.detach()` 를 호출하여 내용물(content)은 같지만, `require_grad`가 다른 새로운 Tensor를 가져옵니다.

In [32]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
